In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from pandas import Series, DataFrame

In [3]:
# analysis html table data, return 3 series
def analysis_table(content):
    soup = BeautifulSoup(content, "lxml")    
    tables = soup.find_all('table')
    
    datas = []
    rows = tables[3].findAll('tr')

    for tr in rows:
        cols = tr.findAll('td')

        for td in cols:
            text = td.find(text = True)
            #print text,
            if text is not None:
                datas.append(text)
    
    name = []
    share = []
    value = []

    index = 0

    for item in datas:
        if ('SOLE' in item and index-7>0):
            name.append(datas[index-7])
            share.append(datas[index-3])
            value.append(datas[index-4])

        index += 1
    
    
    name = Series(name)
    name = name.drop(name.index[0])
    share = Series(share)
    share = share.drop(share.index[0])
    value = Series(value)
    value = value.drop(value.index[0])
    
    
    return name,share,value

In [4]:
# 1. get orginal diagram
orig_url = "https://www.sec.gov/Archives/edgar/data/1510589/000090266417003324/xslForm13F_X01/infotable.xml"

In [5]:
orig_result = requests.get(orig_url)
c = orig_result.content
# can be called only one time
orig_name, orig_share, orig_value = analysis_table(c)

In [7]:
# 2. get after1Q data
af1q_url = "https://www.sec.gov/Archives/edgar/data/1510589/000090266417004351/xslForm13F_X01/infotable.xml"
af1g_result = requests.get(af1q_url)
c_q1 = af1g_result.content
# can be called only one time
af1g_name, af1g_share, af1g_value = analysis_table(c_q1)

In [8]:
# 3. get after2Q data
af2q_url = "https://www.sec.gov/Archives/edgar/data/1510589/000090266418001173/xslForm13F_X01/infotable.xml"
af2g_result = requests.get(af2q_url)
c_q2 = af2g_result.content
# can be called only one time
af2g_name, af2g_share, af2g_value = analysis_table(c_q2)

In [9]:
# 4. get current data
now_url = "https://www.sec.gov/Archives/edgar/data/1510589/000090266418003164/xslForm13F_X01/infotable.xml"
now_result = requests.get(now_url)
c_now = now_result.content
# can be called only one time
now_name, now_share, now_value = analysis_table(c_now)

In [340]:
from __future__ import division
# a:old value, b: new value
def quota(a, b):
    if a > b :
        return "-%.2f%%" % ((float(a-b)/float(a))*100)
    
    if a == b:
        return "0.0%"
    
    if a < b:
        return "+%.2f%%" %((float(b-a)/float(b))*100)

In [342]:
quota(33,33)

'0.0%'

In [289]:
# map ticker name and share or ticker name and map together
def map_list(name, share):
    
    shares = []
    
    for s in share:
        shares.append(cal_value(s))
        
    dict_ticker = Series(shares, index=name)
    
    return dict_ticker

In [290]:
# get globale value for [name: share] [name: value] in different quarter
o_share_list = map_list(orig_name, orig_share)
o_value_list = map_list(orig_name, orig_value)

aq1_share_list = map_list(af1g_name, af1g_share)
aq1_value_list = map_list(af1g_name, af1g_value)

aq2_share_list = map_list(af2g_name, af2g_share)
aq2_value_list = map_list(af2g_name, af2g_value)

n_share_list = map_list(now_name, now_share)
n_value_list = map_list(now_name, now_value)

In [299]:
aq1_share_list.head()

58 COM INC                5550451
ACCELERON PHARMA INC       700000
ALIBABA GROUP HLDG LTD    3570449
AMAZON COM INC              11562
BEIGENE LTD               3896782
dtype: int64

In [300]:
aq2_share_list.head()

58 COM INC                     5550451
ACCELERON PHARMA INC            700000
ALIBABA GROUP HLDG LTD         2999020
AMAZON COM INC                   11562
APELLIS PHARMACEUTICALS INC    1428571
dtype: int64

In [283]:
# conver string to value
def cal_value(value):
    value_list = value.split(',')
    result = ""

    i = 0
    while i < len(value_list):
        result += value_list[i]
        i+=1
    
    return int(result)

In [281]:
cal_value(af1g_share.get(1))

[u'5', u'550', u'451']


5550451

In [284]:
# get venture total value
def get_total_value(orig_value):
    
    index1 = 1
    value1 = 0
    for i in orig_value:
        value1 += cal_value(orig_value.get(index1))
        index1 += 1
        
    return value1

In [285]:
get_total_value(orig_value)

6488017

In [286]:
get_total_value(af1g_value)

6052506

In [287]:
get_total_value(af2g_value)

5890918

In [288]:
get_total_value(now_value)

5867208

In [222]:
sec_report = pd.concat([orig_name, orig_share, orig_value, af1g_name, af1g_share, 
                       af1g_value, af2g_name, af2g_share, af2g_value,now_name,
                       now_share, now_value], axis=1)
sec_report.columns = ['Original Name', 'Original Share', 'Original Value', 
                     'After 1 Quarter Name', 'After 1 Quarter Share', 'After 1 Quarter Value',
                     'After 2 Quarter Name', 'After 2 Quarter Share', 'After 2 Quarter Value',
                     'Current Name', 'Current Share', 'Current Value']

In [223]:
sec_report

,Original Name,Original Share,Original Value,After 1 Quarter Name,After 1 Quarter Share,After 1 Quarter Value,After 2 Quarter Name,After 2 Quarter Share,After 2 Quarter Value,Current Name,Current Share,Current Value
1,58 COM INC,"5,550,451","244,830",58 COM INC,"5,550,451","350,455",58 COM INC,"5,550,451","397,246",58 COM INC,"5,348,673","370,877"
2,AC IMMUNE SA,"550,000","4,840",ACCELERON PHARMA INC,"700,000","26,124",ACCELERON PHARMA INC,"700,000","29,708",ACCELERON PHARMA INC,"700,000","33,964"
3,ACCELERON PHARMA INC,"700,000","21,273",ALIBABA GROUP HLDG LTD,"3,570,449","616,652",ALIBABA GROUP HLDG LTD,"2,999,020","517,121",ALIBABA GROUP HLDG LTD,"6,508,438","1,207,511"
4,ADAPTIMMUNE THERAPEUTICS PLC,"1,190,476","5,345",AMAZON COM INC,"11,562","11,115",AMAZON COM INC,"11,562","13,521",APELLIS PHARMACEUTICALS INC,"4,328,289","95,222"
5,ALIBABA GROUP HLDG LTD,"3,570,449","503,076",BEIGENE LTD,"3,896,782","403,161",APELLIS PHARMACEUTICALS INC,"1,428,571","31,000",ARCUS BIOSCIENCES INC,"767,128","9,390"
6,AMAZON COM INC,"11,562","11,192",BERKSHIRE HATHAWAY INC DEL,64,"17,583",BEIGENE LTD,"3,896,782","380,794",BAIDU INC,"444,831","108,094"
7,BEIGENE LTD,"3,720,726","167,433",BITAUTO HLDGS LTD,"1,124,017","50,221",BERKSHIRE HATHAWAY INC DEL,64,"19,046",BEIGENE LTD,"5,472,259","841,250"
8,BERKSHIRE HATHAWAY INC DEL,60,"15,282",BLUEPRINT MEDICINES CORP,"598,849","41,722",BLUEPRINT MEDICINES CORP,"1,368,449","103,195",BERKSHIRE HATHAWAY INC DEL,51,"14,384"
9,BIOHAVEN PHARMACTL HLDG CO L,"120,000","3,000",BRIGHT SCHOLAR ED HLDGS LTD,"4,300,000","99,889",BRIGHT SCHOLAR ED HLDGS LTD,"4,300,000","80,281",BLUEPRINT MEDICINES CORP,"1,430,949","90,837"
10,BITAUTO HLDGS LTD,"1,124,017","32,315",CAMECO CORP,"1,025,700","9,919",CAMECO CORP,"1,025,700","9,467",BRIGHT SCHOLAR ED HLDGS LTD,"4,293,267","71,397"


In [343]:
# add change percent to series, return new shares and values
def add_cpt(name1, name2, list1, list2, list3, list4):
    shares = []
    values = []
    exist = False
    
    shares.append("")
    values.append("")
    
    for n in name2:
        for m in name1:
            # check if ticker already exist
            if n == m:
                exist = True
                share1 = list1.get(m)
                share2 = list2.get(m)
#                 print(n)
#                 print(share1)
#                 print(share2)
                dis_share = quota(share1, share2)
#                 print("disshare")
#                 print(dis_share)
                shares.append(dis_share)
                
                value1 = list3.get(m)
                value2 = list4.get(m)
                
                dis_value = quota(value1, value2)
                values.append(dis_value)
                break
        
        if exist == False:
            share3 = list2.get(n)
            value4 = list4.get(n)
#             print("share3")
#             print(share3)
            shares.append(share3)
            values.append(value4)
        
        exist = False
        
    
    shares = Series(shares)
    values = Series(values)
    
    return shares, values

In [344]:
af1q_share_cpt, af1q_value_cpt = add_cpt(orig_name, af1g_name, o_share_list, aq1_share_list, o_value_list, aq1_value_list)

In [345]:
af1q_share_cpt.head()

0        
1    0.0%
2    0.0%
3    0.0%
4    0.0%
dtype: object

In [346]:
af2q_share_cpt, af2q_value_cpt = add_cpt(af1g_name, af2g_name, aq1_share_list, aq2_share_list, aq1_value_list,  aq2_value_list)

In [347]:
curr_share_cpt, curr_value_cpt = add_cpt(af2g_name, now_name, aq2_share_list, n_share_list, aq2_value_list,  n_value_list)

In [348]:
sec_report = pd.concat([orig_name, orig_share, orig_value, af1g_name, af1q_share_cpt, 
                       af1q_value_cpt, af2g_name, af2q_share_cpt, af2q_value_cpt,now_name,
                       curr_share_cpt, curr_value_cpt], axis=1)
sec_report.columns = ['Original Name', 'Original Share', 'Original Value', 
                     'After 1 Quarter Name', 'After 1 Quarter Share', 'After 1 Quarter Value',
                     'After 2 Quarter Name', 'After 2 Quarter Share', 'After 2 Quarter Value',
                     'Current Name', 'Current Share', 'Current Value']

In [349]:
sec_report

,Original Name,Original Share,Original Value,After 1 Quarter Name,After 1 Quarter Share,After 1 Quarter Value,After 2 Quarter Name,After 2 Quarter Share,After 2 Quarter Value,Current Name,Current Share,Current Value
0,NaN,NaN,NaN,NaN,,,NaN,,,NaN,,
1,58 COM INC,"5,550,451","244,830",58 COM INC,0.0%,+30.14%,58 COM INC,0.0%,+11.78%,58 COM INC,-3.64%,-6.64%
2,AC IMMUNE SA,"550,000","4,840",ACCELERON PHARMA INC,0.0%,+18.57%,ACCELERON PHARMA INC,0.0%,+12.06%,ACCELERON PHARMA INC,0.0%,+12.53%
3,ACCELERON PHARMA INC,"700,000","21,273",ALIBABA GROUP HLDG LTD,0.0%,+18.42%,ALIBABA GROUP HLDG LTD,-16.00%,-16.14%,ALIBABA GROUP HLDG LTD,+53.92%,+57.17%
4,ADAPTIMMUNE THERAPEUTICS PLC,"1,190,476","5,345",AMAZON COM INC,0.0%,-0.69%,AMAZON COM INC,0.0%,+17.79%,APELLIS PHARMACEUTICALS INC,+66.99%,+67.44%
5,ALIBABA GROUP HLDG LTD,"3,570,449","503,076",BEIGENE LTD,+4.52%,+58.47%,APELLIS PHARMACEUTICALS INC,1428571,31000,ARCUS BIOSCIENCES INC,767128,9390
6,AMAZON COM INC,"11,562","11,192",BERKSHIRE HATHAWAY INC DEL,+6.25%,+13.09%,BEIGENE LTD,0.0%,-5.55%,BAIDU INC,444831,108094
7,BEIGENE LTD,"3,720,726","167,433",BITAUTO HLDGS LTD,0.0%,+35.65%,BERKSHIRE HATHAWAY INC DEL,0.0%,+7.68%,BEIGENE LTD,+28.79%,+54.73%
8,BERKSHIRE HATHAWAY INC DEL,60,"15,282",BLUEPRINT MEDICINES CORP,+2.61%,+29.17%,BLUEPRINT MEDICINES CORP,+56.24%,+59.57%,BERKSHIRE HATHAWAY INC DEL,-20.31%,-24.48%
9,BIOHAVEN PHARMACTL HLDG CO L,"120,000","3,000",BRIGHT SCHOLAR ED HLDGS LTD,0.0%,+48.77%,BRIGHT SCHOLAR ED HLDGS LTD,0.0%,-19.63%,BLUEPRINT MEDICINES CORP,+4.37%,-11.98%
